# IBM capstone project-The battle of Neighborhoods|Finding a Better medical resource and Higer live quality 

## Business Proposal

### Introduction

The purpose of this Project is to help people in exploring better facilities around their neighborhood. It will help people making smart and efficient decision on selecting great neighborhood out of numbers of other neighborhoods in Berlin

Lots of people are migrating to Europe,Germany is one of the top option for them. They needed lots of research for good housing prices and reputated schools for their children. This project is for those people who are looking for better neighborhoods. For ease of accessing to Cafe, School, Super market, medical shops, grocery shops, mall, theatre, hospital, like minded people, etc.

This Project aim to create an analysis of features for a people migrating to Freiburg, compare the neigborhoods of the the city and to search a best neighborhood as a comparative analysis between neighborhoods. The features include median housing price and better school according to ratings, crime rates of that particular area, road connectivity, and medical resources, weather conditions, water resources both freash and waste water and and recreational facilities.

It will help people to get awareness of the area and neighborhood before moving to a new city, state, country or place for their work or to start a new fresh life.

### Problem Which tried to solve

The major purpose of this project, is to suggest a better neighborhood in a new city for the person who are shiffting there and want a great medical resource and high live quality. Social presence in society in terms of like minded people. Connectivity to the airport, bus stand, city center, markets and other daily needs things nearby.

1. Sorted list of house in terms of housing prices in a ascending or descending order
2. Sorted list of hospitals in terms of location, fees, rating and reviews

### The location

Freiburg, is an independent city in Baden-Württemberg, Germany. A famous old German university town, and archiepiscopal seat, Freiburg was incorporated in the early twelfth century and developed into a major commercial, intellectual, and ecclesiastical center of the upper Rhine region. The city is known for its medieval minster and Renaissance university, as well as for its high standard of living and advanced environmental practices. 

### Foursquare API

This project would use Four-square API as its prime data gathering source as it has a database of millions of places, especially their places API which provides the ability to perform location search, location sharing and details about a business.

### Work Flow:

Using credentials of Foursquare API features of near-by places of the neighborhoods would be mined. Due to http request limitations the number of places per neighborhood parameter would reasonably be set to 100 and the radius parameter would be set to 500.

### Clustering Approach:

To explore Freiburg, we explore neighborhoods, segment them, and group them into clusters. To be able to do that, we need to cluster data which is a form of unsupervised machine learning: k-means clustering algorithm

### Libraries Which are Used to Develope the Project:

Pandas: For creating and manipulating dataframes.

Folium: Python visualization library would be used to visualize the neighborhoods cluster distribution of using interactive leaflet map.

Scikit Learn: For importing k-means clustering.

JSON: Library to handle JSON files.

XML: To separate data from presentation and XML stores data in plain text format.

Geocoder: To retrieve Location Data.

Beautiful Soup and Requests: To scrap and library to handle http requests.

Matplotlib: Python Plotting Module.

## Data Description

Data link: https://en.wikipedia.org/wiki/List_of_postal_codes_in_Germany

### Foursquare API Data:

We will need data about different venues in different neighborhoods of that specific borough. In order to gain that information we will use "Foursquare" locational information. Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos. As such, the foursquare location platform will be used as the sole data source since all the stated required information can be obtained through the API.

After finding the list of neighborhoods, we then connect to the Foursquare API to gather information about venues inside each and every neighborhood. For each neighborhood, we have chosen the radius to be 100 meter.

The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:

1. Neighborhood
2. Neighborhood Latitude
3. Neighborhood Longitude
4. Venue
5. Name of the venue e.g. the name of a store or restaurant
6. Venue Latitude
7. Venue Longitude
8. Venue Category

## Methodology Section

### installing and importing python libraries and dependencies

In [1]:
!pip install geocoder
!pip install folium
!pip install sklearn
!pip install geopy
!pip install bs4
!pip install lxml bs4 html5lib

In [2]:
import pandas as pd
import requests
import numpy as np
import geocoder 
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

All Required Libraries Imported!


## Data Extraction and cleaning 

Using the list of Postal Codes of Given Wikipedia Page

In [3]:
Germany_url="https://raw.githubusercontent.com/zauberware/postal-codes-json-xml-csv/master/data/DE/zipcodes.de.json"
Germany=pd.read_json(Germany_url)

In [4]:
Germany.shape

(16481, 11)

In [5]:
Germany.head()

,country_code,zipcode,place,state,state_code,province,province_code,community,community_code,latitude,longitude
0,DE,1945,Grünewald,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4000,14.0000
1,DE,1945,Lindenau,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4000,13.7333
2,DE,1945,Hohenbocka,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4310,14.0098
3,DE,1945,Schwarzbach,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4500,13.9333
4,DE,1945,Guteborn,Brandenburg,BB,,0,Landkreis Oberspreewald-Lausitz,12066,51.4167,13.9333


In [6]:
Germany.drop(['country_code','state_code','province_code','community_code','state'],axis = 'columns', inplace = True)

In [7]:
Germany.rename(columns={"zipcode":"postalcode","place":"neighborhoods","community":"Borough"},inplace=True)

In [8]:
Germany=Germany.dropna()
empty = 'Not assigned'
Germany=Germany[(Germany.postalcode!= empty ) & (Germany.Borough != empty) & (Germany.neighborhoods!= empty)&(Germany.province!=empty)]

In [9]:
Germany.head()

,postalcode,neighborhoods,province,Borough,latitude,longitude
0,1945,Grünewald,,Landkreis Oberspreewald-Lausitz,51.4000,14.0000
1,1945,Lindenau,,Landkreis Oberspreewald-Lausitz,51.4000,13.7333
2,1945,Hohenbocka,,Landkreis Oberspreewald-Lausitz,51.4310,14.0098
3,1945,Schwarzbach,,Landkreis Oberspreewald-Lausitz,51.4500,13.9333
4,1945,Guteborn,,Landkreis Oberspreewald-Lausitz,51.4167,13.9333


In [10]:
Freiburg=Germany[Germany['province']=='Freiburg Region'].reset_index(drop=True)

In [11]:
Freiburg.shape

(327, 6)

In [12]:
Freiburg.head()

,postalcode,neighborhoods,province,Borough,latitude,longitude
0,72172,Sulz am Neckar,Freiburg Region,Rottweil,48.3624,8.6331
1,72175,Dornhan,Freiburg Region,Rottweil,48.3501,8.5090
2,72189,Vöhringen,Freiburg Region,Rottweil,48.3345,8.6639
3,77652,Offenburg,Freiburg Region,Ortenaukreis,48.4765,7.9438
4,77654,Offenburg,Freiburg Region,Ortenaukreis,48.4651,7.9559


## Map of Freiburg

In [13]:
address = 'Freiburg im Breisgau,Freiburg im Breisgau'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude_x = location.latitude
longitude_y = location.longitude
print('The Geograpical Coordinate of Freibrug are {}, {}.'.format(latitude_x, longitude_y))

The Geograpical Coordinate of Freibrug are 47.9975175, 7.8411672.


In [14]:
map_Freiburg =folium.Map(location=[latitude_x, longitude_y], zoom_start=10)

for lat, lng, nei in zip(Freiburg['latitude'], Freiburg['longitude'], Freiburg['neighborhoods']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Freiburg)  
    
map_Freiburg

In [15]:
CLIENT_ID='4DUPT32JSBF4RGHB40QAJJ3SKQG5GA1X52RPJTGUKXU23EHO'
CLIENT_SECRET='WB5CZBEHI43JJLOUJXBXJN53AL11QTLLLP1SRP02GDHG4IQK'
VERSION='20210215'
LIMIT=30
print('Your credentials:')
print('CLIENT_ID:'+CLIENT_ID)
print('CLIENT_SECRET:'+CLIENT_SECRET)

Your credentials:
CLIENT_ID:4DUPT32JSBF4RGHB40QAJJ3SKQG5GA1X52RPJTGUKXU23EHO
CLIENT_SECRET:WB5CZBEHI43JJLOUJXBXJN53AL11QTLLLP1SRP02GDHG4IQK


In [16]:
radius = 700 
LIMIT = 100
url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_x, 
    longitude_y, 
    radius, 
   LIMIT)
results = requests.get(url).json()

In [17]:
Freiburg_venues=results['response']['groups'][0]['items']
Freiburg_nearby_venues = json_normalize(Freiburg_venues)
Freiburg_nearby_venues.columns

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.venuePage.id', 'venue.location.neighborhood',
       'venue.location.crossStreet'],
      dtype='object')

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Freiburg Nearby Venues/Locations

In [19]:
Freiburg_filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Freiburg_nearby_venues =Freiburg_nearby_venues.loc[:, Freiburg_filtered_columns]
Freiburg_nearby_venues.head()


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Hermann,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",47.996466,7.839550
1,Sofra Döner,"[{'id': '4bf58dd8d48988d10b941735', 'name': 'F...",47.996159,7.842642
2,Cafe Einstein,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",47.998694,7.839642
3,Ochsebrugg,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",47.997777,7.840047
4,curry house,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",47.999465,7.838838


In [20]:
Freiburg_nearby_venues['venue.categories'] =Freiburg_nearby_venues.apply(get_category_type, axis=1)

# clean columns
Freiburg_nearby_venues.columns = [col.split(".")[-1] for col in Freiburg_nearby_venues.columns]

Freiburg_nearby_venues.head(5)

,name,categories,lat,lng
0,Hermann,Café,47.996466,7.839550
1,Sofra Döner,Falafel Restaurant,47.996159,7.842642
2,Cafe Einstein,Café,47.998694,7.839642
3,Ochsebrugg,Pizza Place,47.997777,7.840047
4,curry house,Indian Restaurant,47.999465,7.838838


In [21]:
a=pd.Series(Freiburg_nearby_venues.categories)
a.value_counts()[:10]

Café                         20
Hotel                         9
Bar                           5
Nightclub                     4
Cocktail Bar                  4
Supermarket                   3
Middle Eastern Restaurant     3
Italian Restaurant            3
Asian Restaurant              2
Bank                          2
Name: categories, dtype: int64

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # making GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
# Nearby Venues
Freiburg_venues = getNearbyVenues(names=Freiburg['neighborhoods'],
                                   latitudes=Freiburg['latitude'],
                                   longitudes=Freiburg['longitude']
                                  )

Sulz am Neckar
Dornhan
Vöhringen
Offenburg
Offenburg
Offenburg
Kehl
Oberkirch
Oberwolfach
Wolfach
Haslach im Kinzigtal
Hofstetten
Fischerbach
Gengenbach
Oppenau
Willstätt
Zell am Harmersbach
Bad Peterstal-Griesbach
Neuried
Schutterwald
Hohberg
Hausach
Schiltach
Appenweier
Durbach
Schenkenzell
Biberach
Oberharmersbach
Nordrach
Steinach
Berghaupten
Gutach (Schwarzwaldbahn)
Lautenbach
Mühlenbach
Ohlsbach
Ortenberg
Achern
Rheinau
Renchen
Kappelrodeck
Sasbach
Ottenhöfen im Schwarzwald
Lauf
Sasbachwalden
Seebach
Lahr/Schwarzwald
Friesenheim
Friesenheim
Ettenheim
Seelbach
Schwanau
Kappel-Grafenhausen
Kippenheim
Mahlberg
Meißenheim
Ringsheim
Rust
Schuttertal
Villingen-Schwenningen
Villingen-Schwenningen
Villingen-Schwenningen
Villingen-Schwenningen
Villingen-Schwenningen
Bad Dürrheim
Niedereschach
Dauchingen
Brigachtal
Mönchweiler
Unterkirnach
Triberg
Sankt Georgen im Schwarzwald
Furtwangen im Schwarzwald
Königsfeld im Schwarzwald
Hornberg
Schonach im Schwarzwald
Schönwald im Schwarzwald
Tenne

In [24]:
print('There are {} Uniques Categories.'.format(len(Freiburg_venues['Venue Category'].unique())))
Freiburg_venues.groupby('Neighborhood').count().head()

There are 243 Uniques Categories.


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aach,3,3,3,3,3,3
Achern,5,5,5,5,5,5
Aichhalden,3,3,3,3,3,3
Aitern,1,1,1,1,1,1
Albbruck,5,5,5,5,5,5


## One Hot Encoding of Features

In [25]:
# one hot encoding
Freiburg_onehot = pd.get_dummies(Freiburg_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Freiburg_onehot['Neighborhood'] = Freiburg_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Freiburg_onehot.columns[-1]] + list(Freiburg_onehot.columns[:-1])
Freiburg_onehot = Freiburg_onehot[fixed_columns]
Freiburg_grouped = Freiburg_onehot.groupby('Neighborhood').mean().reset_index()
Freiburg_onehot.head(5)

,Neighborhood,ATM,Advertising Agency,Airport,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cable Car,Café,Campground,Candy Store,Castle,Cave,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Dairy Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Spot,Doctor's Office,Doner Restaurant,Drugstore,Electronics Store,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flower Shop,Food,Food & Drink Shop,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Lake,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Massage Studio,Mattress Store,Medical Supply Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Moroccan Restaurant,Motel,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Venue,National Park,Nightclub,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Outlet Mall,Palace,Park,Parking,Pastry Shop,Pedestrian Plaza,Pet Store,Pharmacy,Photography Studio,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Print Shop,Pub,Public Art,RV Park,Racetrack,Recycling Facility,Rental Car Location,Rental Service,Resort,Rest Area,Restaurant,Rhenisch Restaurant,River,Rock Climbing Spot,Salad Place,Sandwich Place,Sauna / Steam Room,Scenic Lookout,Scottish Restaurant,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Ski Area,Ski Chairlift,Ski Trail,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stables,Steakhouse,Storage Facility,Summer Camp,Supermarket,Sushi Restaurant,Swabian Restaurant,Swiss Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Taverna,Thai Restaurant,Theater,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train,Train Station,Tram Station,Trattoria/Osteria,Travel Agency,Tree,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Vineyard,Water Park,Waterfall,Wine Bar,Wine Shop,Women's Store
0,Sulz am Neckar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Sulz am Neckar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [26]:
num_top_venues = 5
for hood in Freiburg_grouped['Neighborhood']:
    print("---- "+hood+" ----")
    temp =Freiburg_grouped[Freiburg_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Aach ----
               venue  freq
0                ATM  0.33
1               Lake  0.33
2         Public Art  0.33
3        Planetarium  0.00
4  Outdoor Sculpture  0.00


---- Achern ----
                venue  freq
0              Bakery   0.2
1                Café   0.2
2          Steakhouse   0.2
3         Supermarket   0.2
4  Italian Restaurant   0.2


---- Aichhalden ----
               venue  freq
0   Kebab Restaurant  0.33
1  German Restaurant  0.33
2              Hotel  0.33
3        Planetarium  0.00
4  Outdoor Sculpture  0.00


---- Aitern ----
                   venue  freq
0     Chinese Restaurant   1.0
1                    ATM   0.0
2   Other Great Outdoors   0.0
3      Outdoor Sculpture   0.0
4  Outdoors & Recreation   0.0


---- Albbruck ----
                        venue  freq
0                  Restaurant   0.2
1          Rock Climbing Spot   0.2
2               Train Station   0.2
3  Construction & Landscaping   0.2
4                 Pizza Place   0.2


---- Al

              venue  freq
0  Insurance Office   0.4
1     Grocery Store   0.2
2           Stables   0.2
3            Garden   0.2
4       Pizza Place   0.0


---- Efringen-Kirchen ----
                        venue  freq
0           German Restaurant   0.4
1                 Supermarket   0.2
2  Construction & Landscaping   0.2
3               Train Station   0.2
4          Recycling Facility   0.0


---- Egesheim ----
               venue  freq
0              Trail   1.0
1                ATM   0.0
2        Pizza Place   0.0
3  Other Repair Shop   0.0
4  Outdoor Sculpture   0.0


---- Eggingen ----
                   venue  freq
0      German Restaurant   1.0
1                    ATM   0.0
2   Other Great Outdoors   0.0
3      Outdoor Sculpture   0.0
4  Outdoors & Recreation   0.0


---- Ehrenkirchen ----
               venue  freq
0             Bakery  0.25
1        Supermarket  0.25
2   Business Service  0.25
3               Bank  0.25
4  Outdoor Sculpture  0.00


---- Eigeltingen ---

4  Outdoor Sculpture   0.0


---- Hausen ob Verena ----
                   venue  freq
0      German Restaurant   1.0
1                    ATM   0.0
2   Other Great Outdoors   0.0
3      Outdoor Sculpture   0.0
4  Outdoors & Recreation   0.0


---- Heitersheim ----
                venue  freq
0         Supermarket  0.33
1               Hotel  0.33
2   German Restaurant  0.17
3    Storage Facility  0.17
4  Recycling Facility  0.00


---- Herbolzheim ----
                venue  freq
0         Supermarket  0.50
1  Italian Restaurant  0.25
2    Insurance Office  0.25
3                Park  0.00
4            Pharmacy  0.00


---- Herrischried ----
                        venue  freq
0  Construction & Landscaping   1.0
1                         ATM   0.0
2                 Planetarium   0.0
3           Outdoor Sculpture   0.0
4       Outdoors & Recreation   0.0


---- Hilzingen ----
               venue  freq
0        Supermarket   0.4
1   Insurance Office   0.2
2              Hotel   0.2
3  

4                         ATM  0.00


---- Meißenheim ----
                  venue  freq
0           Supermarket  0.67
1           IT Services  0.33
2                   ATM  0.00
3    Photography Studio  0.00
4  Other Great Outdoors  0.00


---- Merdingen ----
               venue  freq
0        Supermarket  0.33
1  Electronics Store  0.33
2           Vineyard  0.33
3        Pizza Place  0.00
4  Other Repair Shop  0.00


---- Merzhausen ----
                venue  freq
0              Bakery  0.50
1        Concert Hall  0.25
2         Supermarket  0.25
3         Pizza Place  0.00
4  Photography Studio  0.00


---- Moos ----
                        venue  freq
0                 Supermarket   0.2
1  Construction & Landscaping   0.2
2                       Beach   0.2
3           German Restaurant   0.2
4            Insurance Office   0.2


---- Murg ----
                    venue  freq
0      Sauna / Steam Room  0.25
1      Italian Restaurant  0.25
2           Train Station  0.25
3  Furni

           venue  freq
0            Gym  0.25
1  Train Station  0.25
2      Gastropub  0.25
3           Tree  0.25
4        Parking  0.00


---- Sallneck ----
                   venue  freq
0      German Restaurant   1.0
1                    ATM   0.0
2   Other Great Outdoors   0.0
3      Outdoor Sculpture   0.0
4  Outdoors & Recreation   0.0


---- Sankt Blasien ----
            venue  freq
0     Supermarket  0.33
1           Hotel  0.22
2  Ice Cream Shop  0.11
3     Pizza Place  0.11
4             Gym  0.11


---- Sankt Georgen im Schwarzwald ----
               venue  freq
0        Supermarket  0.18
1  German Restaurant  0.09
2              Hotel  0.09
3             Bakery  0.09
4      Train Station  0.09


---- Sankt Märgen ----
               venue  freq
0              Hotel  0.50
1                ATM  0.25
2               Café  0.25
3        Planetarium  0.00
4  Outdoor Sculpture  0.00


---- Sankt Peter ----
                        venue  freq
0                       Hotel  0.17

                   venue  freq
0      German Restaurant   1.0
1                    ATM   0.0
2   Other Great Outdoors   0.0
3      Outdoor Sculpture   0.0
4  Outdoors & Recreation   0.0


---- Tuningen ----
                        venue  freq
0                 Supermarket   0.5
1  Construction & Landscaping   0.5
2                 Planetarium   0.0
3           Other Repair Shop   0.0
4           Outdoor Sculpture   0.0


---- Tuttlingen ----
                venue  freq
0      Clothing Store  0.09
1  Italian Restaurant  0.09
2               Hotel  0.06
3               Plaza  0.06
4                Café  0.06


---- Umkirch ----
              venue  freq
0       Supermarket   0.4
1  Asian Restaurant   0.2
2        Restaurant   0.2
3             Hotel   0.2
4          Pharmacy   0.0


---- Unterkirnach ----
               venue  freq
0              Hotel  0.25
1  German Restaurant  0.25
2          BBQ Joint  0.25
3         Playground  0.25
4        Pastry Shop  0.00


---- Utzenfeld ----
 

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Freiburg_grouped['Neighborhood']

for ind in np.arange(Freiburg_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Freiburg_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aach,ATM,Lake,Public Art,Food,General Entertainment,Gastropub,Gas Station,Garden Center,Garden,Furniture / Home Store
1,Achern,Italian Restaurant,Steakhouse,Supermarket,Café,Bakery,General Travel,General Entertainment,Gastropub,Gas Station,Garden Center
2,Aichhalden,German Restaurant,Kebab Restaurant,Hotel,Women's Store,Fried Chicken Joint,Forest,Fountain,Frame Store,French Restaurant,Furniture / Home Store
3,Aitern,Chinese Restaurant,German Restaurant,General Entertainment,Gastropub,Gas Station,Garden Center,Garden,Furniture / Home Store,Fried Chicken Joint,French Restaurant
4,Albbruck,Pizza Place,Restaurant,Train Station,Rock Climbing Spot,Construction & Landscaping,Frame Store,Food,Food & Drink Shop,Forest,Fountain


## K-Means Clustering Approach 

In [29]:
Freiburg_grouped_clustering = Freiburg_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(Freiburg_grouped_clustering)
kmeans.labels_

array([2, 0, 4, 2, 2, 2, 0, 0, 2, 2, 4, 4, 0, 4, 2, 4, 2, 0, 0, 0, 4, 2,
       2, 4, 2, 4, 2, 0, 0, 3, 0, 4, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 1, 2,
       2, 0, 0, 0, 2, 2, 4, 2, 2, 2, 3, 2, 3, 0, 2, 2, 4, 2, 2, 2, 0, 2,
       2, 2, 0, 2, 2, 2, 2, 3, 2, 4, 4, 2, 4, 0, 2, 0, 0, 2, 3, 0, 2, 0,
       1, 0, 0, 2, 2, 0, 2, 2, 2, 2, 4, 0, 2, 2, 3, 0, 0, 1, 0, 2, 3, 2,
       0, 1, 2, 2, 0, 4, 2, 1, 2, 2, 2, 2, 0, 2, 0, 0, 2, 4, 4, 2, 3, 2,
       0, 3, 1, 0, 2, 2, 2, 4, 4, 4, 2, 2, 2, 2, 2, 2, 4, 3, 2, 0, 0, 0,
       0, 2, 2, 2, 2, 0, 2, 2, 2, 4, 2, 0, 2, 0, 2, 4, 2, 2, 3, 4, 2, 4,
       2, 4, 0, 4, 4, 2, 2, 0, 2, 0, 0, 2, 4, 4, 2, 0, 4, 2, 4, 2, 2, 3,
       0, 0, 4, 2, 3, 4, 4, 2, 4, 2, 2, 0, 4, 2, 0, 2, 2, 0, 2, 0, 4, 2,
       4, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 4, 0, 2, 4, 2, 0, 2, 0, 4, 0, 4,
       0, 2, 2, 3, 1, 2, 0, 4, 2, 0, 2, 2, 0, 2, 1, 0, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 0, 4, 2, 3, 0, 2, 2, 2, 0, 0, 2, 2, 0, 1], dtype=int32)

In [30]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Freiburg_merged =Freiburg.iloc[:16,:]

# merge Freiburg_grouped with Freiburg_data to add latitude/longitude for each neighborhood
Freiburg_merged =Freiburg_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='neighborhoods')

Freiburg_merged.head()# check the last columns!

,postalcode,neighborhoods,province,Borough,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,72172,Sulz am Neckar,Freiburg Region,Rottweil,48.3624,8.6331,0,Shoe Store,Supermarket,Train Station,Drugstore,French Restaurant,Food & Drink Shop,Forest,Fountain,Frame Store,Furniture / Home Store
1,72175,Dornhan,Freiburg Region,Rottweil,48.3501,8.5090,2,Construction & Landscaping,Furniture / Home Store,Automotive Shop,Women's Store,Food & Drink Shop,General Travel,General Entertainment,Gastropub,Gas Station,Garden Center
2,72189,Vöhringen,Freiburg Region,Rottweil,48.3345,8.6639,1,Construction & Landscaping,Supermarket,Furniture / Home Store,Food,General Travel,General Entertainment,Gastropub,Gas Station,Garden Center,Garden
3,77652,Offenburg,Freiburg Region,Ortenaukreis,48.4765,7.9438,2,Supermarket,Café,Platform,Bakery,Hotel,Concert Hall,Fast Food Restaurant,Nightclub,Brewery,Gym
4,77654,Offenburg,Freiburg Region,Ortenaukreis,48.4651,7.9559,2,Supermarket,Café,Platform,Bakery,Hotel,Concert Hall,Fast Food Restaurant,Nightclub,Brewery,Gym


## Map of Clusters

In [31]:
kclusters =5

In [43]:
# create map
map_clusters = folium.Map(location=[latitude_x, longitude_y], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map

markers_colors = []
for lat, lon, nei , cluster in zip(Freiburg_merged['latitude'], 
                                   Freiburg_merged['longitude'], 
                                   Freiburg_merged['neighborhoods'], 
                                   Freiburg_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [37]:
df1=Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 0,Freiburg_merged.columns[[2] + list(range(5, Freiburg_merged.shape[1]))]]
df2=Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 1,Freiburg_merged.columns[[2] + list(range(5, Freiburg_merged.shape[1]))]]
df3=Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 2,Freiburg_merged.columns[[2] + list(range(5, Freiburg_merged.shape[1]))]]
df4=Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 3,Freiburg_merged.columns[[2] + list(range(5, Freiburg_merged.shape[1]))]]
df5=Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 4,Freiburg_merged.columns[[2] + list(range(5, Freiburg_merged.shape[1]))]]

## Examing The Clusters

In [38]:
Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 0, Freiburg_merged.columns[[1] + list(range(5, Freiburg_merged.shape[1]))]]

,neighborhoods,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Sulz am Neckar,8.6331,0,Shoe Store,Supermarket,Train Station,Drugstore,French Restaurant,Food & Drink Shop,Forest,Fountain,Frame Store,Furniture / Home Store
9,Wolfach,8.2158,0,German Restaurant,Supermarket,Train Station,Swabian Restaurant,History Museum,Gas Station,Gastropub,Garden Center,Food & Drink Shop,Furniture / Home Store
10,Haslach im Kinzigtal,8.0894,0,Supermarket,Drugstore,Café,Mobile Phone Shop,Women's Store,Forest,General Entertainment,Gastropub,Gas Station,Garden Center
15,Willstätt,7.8931,0,Supermarket,Gastropub,Café,Bank,Women's Store,Golf Course,General Travel,General Entertainment,Gas Station,Garden Center


In [39]:
Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 1, Freiburg_merged.columns[[1] + list(range(5, Freiburg_merged.shape[1]))]]

,neighborhoods,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Vöhringen,8.6639,1,Construction & Landscaping,Supermarket,Furniture / Home Store,Food,General Travel,General Entertainment,Gastropub,Gas Station,Garden Center,Garden


In [40]:
Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 2, Freiburg_merged.columns[[1] + list(range(5, Freiburg_merged.shape[1]))]]

,neighborhoods,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Dornhan,8.5090,2,Construction & Landscaping,Furniture / Home Store,Automotive Shop,Women's Store,Food & Drink Shop,General Travel,General Entertainment,Gastropub,Gas Station,Garden Center
3,Offenburg,7.9438,2,Supermarket,Café,Platform,Bakery,Hotel,Concert Hall,Fast Food Restaurant,Nightclub,Brewery,Gym
4,Offenburg,7.9559,2,Supermarket,Café,Platform,Bakery,Hotel,Concert Hall,Fast Food Restaurant,Nightclub,Brewery,Gym
5,Offenburg,7.9313,2,Supermarket,Café,Platform,Bakery,Hotel,Concert Hall,Fast Food Restaurant,Nightclub,Brewery,Gym
6,Kehl,7.8152,2,Supermarket,Drugstore,Ice Cream Shop,Hotel,Bakery,Restaurant,Clothing Store,Fast Food Restaurant,Café,Shoe Store
7,Oberkirch,8.0786,2,Café,German Restaurant,Hotel,Park,Supermarket,Grocery Store,Historic Site,Dessert Shop,IT Services,Flower Shop
12,Fischerbach,8.1096,2,Comfort Food Restaurant,German Restaurant,Scenic Lookout,Food,General Entertainment,Gastropub,Gas Station,Garden Center,Garden,Furniture / Home Store
13,Gengenbach,8.0143,2,German Restaurant,Hotel,Ice Cream Shop,Plaza,Coffee Shop,Food,Supermarket,Furniture / Home Store,Garden,Fried Chicken Joint
14,Oppenau,8.1597,2,Supermarket,Bakery,Shoe Store,Automotive Shop,Train Station,Café,Bar,Snack Place,Men's Store,Electronics Store


In [41]:
Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 3, Freiburg_merged.columns[[1] + list(range(5, Freiburg_merged.shape[1]))]]

,neighborhoods,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Hofstetten,8.066,3,German Restaurant,Dive Spot,Food & Drink Shop,General Travel,General Entertainment,Gastropub,Gas Station,Garden Center,Garden,Furniture / Home Store


In [42]:
Freiburg_merged.loc[Freiburg_merged['Cluster Labels'] == 4, Freiburg_merged.columns[[1] + list(range(5, Freiburg_merged.shape[1]))]]

,neighborhoods,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Oberwolfach,8.2167,4,Hotel,Pharmacy,German Restaurant,Women's Store,Food & Drink Shop,General Entertainment,Gastropub,Gas Station,Garden Center,Garden


## Results and discussion

The neighbourhoods of Freiburg are not very multicultural. The most resteruants are German resteruant. Freiburg seems to have lots of shopping options like supermarket,plaza,shoppingmall and outletmall. It also has a lot of entertainment options too,like night clubs and bar.The main modes of transport seem to be Buses and trains. For leisure, the neighbourhoods are set up to have lots of parks,concert hell, and Historic sites. Overall, the city of London offers a multicultural, diverse and certainly entertaining experience.

## Conclusion

The purpose of this project was to explore the city of Freiburg and see how attractive it is to potential tourists and migrants. We explored both the city based on their postal codes and then extrapolated the common venues present in each of the neighbourhoods finally concluding with clustering similar neighbourhoods together.
We could see that each of the neighbourhoods in the city have a wide variety of experiences to offer which is unique in its own way. The cultural diversity is not quite evident, but Freiburg seem to offer a vacation stay or a romantic getaway with a lot of places to explore, beautiful landscapes, amazing food and a wide variety of culture. Overall, it’s up-to-the stakeholders to decide which experience they would prefer more and which would more to their liking.
